# Vision Transformer (ViT-Tiny) Fine-tuning 예제

이 노트북에서는 **입력 크기 33×33**인 MC 히트맵 이미지를 **Vision Transformer (ViT-Tiny)** 모델로 분류하는 과정을 단계별로 다룹니다.

**이 예제의 목표**
1. YAML 설정 파일을 활용해 하이퍼파라미터를 관리한다.
2. `torch.utils.data.Dataset`을 활용해 JetHeatmapDataset을 구현한다.
3. 사전학습(pretrained) ViT-Tiny 모델을 로드하고, 블록 동결(partial fine-tuning) 옵션을 적용한다.
4. 학습/검증 루프를 직접 구현하여 Loss와 Accuracy를 모니터링한다.
5. 최종 모델을 저장하고, 새로운 데이터에 대해 추론하는 방법을 확인한다.

각 단계마다 **코드 셀**을 단계별로 구성하였으며, 실행 흐름을 따라가며 ViT fine-tuning 파이프라인을 한눈에 파악해 보세요.

### vit-tiny.yaml 생성 스크립트

- **용도**  
  ImageNet 사전학습(pretrained)된 ViT-Tiny 모델을 불러와 일부 블록만 fine-tuning할 때 사용

- **주요 설정**  
  - `model.embed_dim = 192`, `model.depth = 12` (원본 ViT-Tiny 구조)  
  - `model.finetune_type = "full"` → 전체 블록 학습  
  - `model.finetune_type = "partial"` → 뒤쪽 `model.num_trainable_blocks = 2` 블록만 학습  
  - `optimizer.name = "AdamW"`, `optimizer.lr = 1e-3`, `optimizer.weight_decay = 0.01`  
  - `scheduler.name = "Linear"` with warmup 2 에포크  
  - `transform.resize = True`, `transform.normalize = True` (mean/std = 0.5)

- **사용 방법**  
  1. 스크립트를 실행하면 `/content/drive/MyDrive/예제/vit-tiny.yaml` 파일이 생성됨  
  2. 노트북 실행 전 해당 YAML을 변경하지 말 것  
  3. `model.finetune_type`만 “full” 또는 “partial”로 변경하여 학습 블록 수 조절 가능  

In [ ]:
import yaml
from pathlib import Path

# 1) 프로젝트 루트 경로
BASE = Path(r"/content/drive/MyDrive/예제")


# 2) yaml에 저장할 하이퍼파라미터 설정
vit_tiny_cfg = {
    "epochs": 10,
    "batch_size": 128,
    "num_workers": 0,

    "model": {
        "name": "vit_tiny_patch16_224",
        "patch_size": 16,
        "img_size": 224,
        "embed_dim": 192,
        "depth": 12,
        "num_heads": 2,
        "mlp_ratio": 4.0,
        "qkv_bias": True,
        "drop_rate": 0.1,
        "attn_drop_rate": 0.1,
        "drop_path_rate": 0.1,
        "num_classes": 2,
        "finetune_type": "full",       # "full" 또는 "partial"
        "num_trainable_blocks": 2
    },

    "optimizer": {
        "name": "AdamW",
        "lr": 1e-3,
        "weight_decay": 0.01
    },

    "scheduler": {
        "name": "Linear",                # 예: "StepLR", "Cosine", "Linear"
        "total_epochs": 10,
        "warmup_epochs": 2
    },

    "transform": {
        "resize": True,
        "random_rotation": 0,
        "horizontal_flip": False,
        "vertical_flip": False,
        "normalize": True,
        "mean": [0.5, 0.5, 0.5],
        "std":  [0.5, 0.5, 0.5]
    },

}

# 3) YAML 파일 저장
outfile = BASE / "vit-tiny.yaml"
with open(outfile, "w", encoding="utf-8") as f:
    yaml.dump(vit_tiny_cfg, f, default_flow_style=False, sort_keys=False)

print(f"✅ YAML 설정 파일 저장 완료: {outfile}")

✅ YAML 설정 파일 저장 완료: /content/drive/MyDrive/예제/vit-tiny.yaml


### vit-tiny-custom.yaml 생성 스크립트

- **용도**  
  사전학습 가중치 없이, 경량화된 ViT-Tiny 구조를 처음부터 학습할 때 사용

- **주요 설정**  
  - `model.embed_dim = 128`, `model.depth = 6` (절반 깊이의 경량화 구조)  
  - `model.finetune_type = "full"` → 전체 블록 학습  
  - `optimizer` 및 `scheduler` 설정은 `vit-tiny.yaml`과 동일  

- **사용 방법**  
  1. 스크립트를 실행하면 `/content/drive/MyDrive/예제/vit-tiny-custom.yaml` 파일이 생성됨  
  2. 데이터셋 경로 및 하이퍼파라미터를 프로젝트에 맞게 수정  


In [ ]:
import yaml
from pathlib import Path

# 1) 프로젝트 루트 경로
BASE = Path(r"/content/drive/MyDrive/예제")


# 2) yaml에 저장할 하이퍼파라미터 설정
vit_tiny_cfg = {
    "epochs": 10,
    "batch_size": 128, #128
    "num_workers": 2,

    "model": {
        "name": "vit_tiny_patch16_224",
        "patch_size": 2,
        "img_size": 33,
        "embed_dim": 128, #128
        "depth":6, #6
        "num_heads": 2,
        "mlp_ratio": 2.0,
        "qkv_bias": True,
        "drop_rate": 0.1,
        "attn_drop_rate": 0.1,
        "drop_path_rate": 0.1,
        "num_classes": 2,
        "finetune_type": "full",       # "full" 또는 "partial"
        "num_trainable_blocks": 2
    },

    "optimizer": {
        "name": "AdamW",
        "lr": 1e-3,
        "weight_decay": 0.01
    },

    "scheduler": {
        "name": "Linear",                # 예: "StepLR", "Cosine", "Linear"
        "total_epochs": 10,
        "warmup_epochs": 2
    },

    "transform": {
        "resize": True,
        "random_rotation": 0,
        "horizontal_flip": False,
        "vertical_flip": False,
        "normalize": True,
        "mean": [0.5, 0.5, 0.5],
        "std":  [0.5, 0.5, 0.5]
    },

}

# 3) YAML 파일 저장
outfile = BASE / "vit-tiny-custom.yaml"
with open(outfile, "w", encoding="utf-8") as f:
    yaml.dump(vit_tiny_cfg, f, default_flow_style=False, sort_keys=False)

print(f"✅ YAML 설정 파일 저장 완료: {outfile}")

✅ YAML 설정 파일 저장 완료: /content/drive/MyDrive/예제/vit-tiny-custom.yaml


## 1. 환경 설정

먼저 필요한 라이브러리를 설치 및 로드하고, **GPU가 사용 가능한지** 확인합니다.


In [ ]:
# 1) 핵심 라이브러리 불러오기
import os                   # 파일 시스템 경로 조작
import yaml                 # YAML 설정 파일 파싱
import pandas as pd         # CSV 읽기/쓰기
from PIL import Image       # 이미지 입출력
from sklearn.model_selection import train_test_split  # 데이터 분할
from tqdm import tqdm       # 진행률 표시
import torch                # PyTorch 메인 모듈
import torch.nn as nn       # 신경망 구성 요소
import torch.optim as optim # 최적화 알고리즘
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms  # 이미지 전처리
import timm                 # 사전학습된 Vision Transformer 로드

# 2) 디바이스 설정: GPU 사용 권장
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")  # GPU/CPU 정보 출력


Device: cuda


## 2. JetHeatmapDataset 클래스 구현

이 셀에서는 **CSV** 파일(`labels.csv`)과 **이미지 디렉토리** 경로를 연결하여, 데이터셋을 **샘플 단위**로 로딩하는 클래스를 정의합니다.

- **`__init__`** 메서드:
  - `csv_path`: 파일명과 레이블이 저장된 CSV 경로
  - `img_dir`: 이미지 파일들이 위치한 폴더 경로
  - `transform`: 전처리(transform) 파이프라인 객체
- **`__len__`** 메서드:
  - 데이터셋의 총 샘플 수 반환 (len(self.df))
- **`__getitem__`** 메서드:
  1. DataFrame에서 `fname, label` 추출
  2. `PIL.Image.open`으로 이미지 로드 후 RGB 변환
  3. 지정된 `transform` 적용 (예: Resize, ToTensor, Normalize)
  4. `(tensor_image, int(label))` 튜플 반환

이 클래스를 통해 DataLoader가 **효율적으로 배치 단위**로 이미지를 로드할 수 있습니다.

In [ ]:
class JetHeatmapDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.df = pd.read_csv(csv_path)       # (N, 2) shape: ['filename', 'label']
        self.img_dir = img_dir                # 이미지 디렉토리 경로
        self.transform = transform            # torchvision transforms

    def __len__(self):
        return len(self.df)                  # 전체 샘플 수 반환

    def __getitem__(self, idx):
        fname, label = self.df.iloc[idx]     # 파일명, 레이블 추출
        path = os.path.join(self.img_dir, fname)
        img = Image.open(path).convert('RGB') # RGB 이미지로 변환
        if self.transform:
            img = self.transform(img)        # 전처리 적용
        return img, int(label)              # 이미지 텐서, 레이블 반환


## 3. 이미지 전처리(Transforms) 함수 정의

`vit-tiny.yaml` 설정 파일의 `transform` 섹션에 정의된 옵션에 맞춰, **학습용(train)**과 **검증용(eval)** 파이프라인을 구성합니다.

1. **Resize**: 입력 이미지를 `(img_size × img_size)`로 일괄 크기 조정(예: 33×33)  
2. **RandomHorizontalFlip / RandomVerticalFlip / RandomRotation**: 학습 시에만 적용하여 데이터 증강(data augmentation) 수행  
3. **ToTensor**: PIL 이미지를 PyTorch 텐서로 변환 (값 범위 [0,1])  
4. **Normalize**: 지정된 `mean`, `std`로 표준화 (값 분포 안정화)

이러한 전처리를 통해 학습 안정성과 일반화 성능을 동시에 확보할 수 있습니다.

In [ ]:
def build_transforms(cfg, train=True):
    ops = []
    # 1) Resize (항상)
    if cfg['transform']['resize']:
        size = cfg['model']['img_size']
        ops.append(transforms.Resize((size, size)))
    # 2) 학습 시에만 데이터 증강
    if train:
        if cfg['transform']['horizontal_flip']:
            ops.append(transforms.RandomHorizontalFlip())
        if cfg['transform']['vertical_flip']:
            ops.append(transforms.RandomVerticalFlip())
        if cfg['transform']['random_rotation'] > 0:
            ops.append(transforms.RandomRotation(cfg['transform']['random_rotation']))
    # 3) 텐서 변환 및 정규화
    ops.append(transforms.ToTensor())
    if cfg['transform']['normalize']:
        ops.append(transforms.Normalize(cfg['transform']['mean'], cfg['transform']['std']))
    return transforms.Compose(ops)


## 4. 모델 로드 및 Fine-tuning 설정

`build_model_components(cfg, device, ckpt_path)` 함수가 내부에서 수행하는 작업은 다음과 같습니다:

1. **모델 생성**  
   - `timm.create_model(..., pretrained=False)` 로 ViT-Tiny 구조 초기화  
   - `img_size`, `patch_size`, `embed_dim`, `depth`, `num_heads`, `mlp_ratio`, `drop_rate`, `attn_drop_rate`, `drop_path_rate`, `num_classes` 등 아키텍처 하이퍼파라미터를 YAML 설정에서 불러옵니다.

2. **체크포인트 로드**  
   - `torch.load(ckpt_path)` 로 저장된 `.pth` 파일을 읽어들입니다.  
   - `model_state_dict`, `state_dict` 키가 없을 때를 대비해 `ckpt.get("model_state_dict", ckpt.get("state_dict", ckpt))` 방식으로 안전하게 가중치를 불러와 `model.load_state_dict()` 합니다.

3. **Fine-tuning 모드 적용**  
   - `finetune_type='full'`: 모델의 모든 파라미터를 업데이트 (Full fine-tuning)  
   - `finetune_type='partial'`: 앞쪽 블록을 freeze(동결)하고, 마지막 N개 블록 및 classification head만 학습 (Partial fine-tuning)

4. **Optimizer & Loss 설정**  
   - `AdamW` 옵티마이저 (`lr`, `weight_decay`는 YAML 설정값 사용)  
   - `CrossEntropyLoss` (2-class 분류용)

5. **학습률 스케줄러 (LambdaLR + Warmup)**  
   - 처음 `warmup_epochs` 동안 학습률을 선형 증가  
   - 이후 남은 에포크 동안 학습률을 선형 감소

이 과정을 통해 반환된 `model`, `optimizer`, `criterion`, `scheduler`를 학습 루프에서 바로 사용할 수 있습니다.  


In [ ]:
def build_model_components(cfg, device, ckpt_path):
    # 1) 모델 생성
    m = cfg["model"]
    model = timm.create_model(
        m["name"],
        pretrained=False,
        img_size=m["img_size"],
        patch_size=m["patch_size"],
        embed_dim=m["embed_dim"],
        depth=m["depth"],
        num_heads=m["num_heads"],
        mlp_ratio=m["mlp_ratio"],
        qkv_bias=m["qkv_bias"],
        drop_rate=m["drop_rate"],
        attn_drop_rate=m["attn_drop_rate"],
        drop_path_rate=m["drop_path_rate"],
        num_classes=m["num_classes"]
    ).to(device)

    # 2) 체크포인트 로드 (KeyError 방지용 fallback)
    ckpt = torch.load(ckpt_path, map_location=device)
    state_dict = ckpt.get("model_state_dict",
                 ckpt.get("state_dict",
                          ckpt))
    model.load_state_dict(state_dict)

    # 3) partial fine-tuning 처리
    if m["finetune_type"] == "partial":
        num_freeze = len(model.blocks) - m["num_trainable_blocks"]
        for blk in model.blocks[:num_freeze]:
            for p in blk.parameters():
                p.requires_grad = False

    # 4) 옵티마이저 & 손실함수
    optimizer = optim.AdamW(
        model.parameters(),
        lr=cfg["optimizer"]["lr"],
        weight_decay=cfg["optimizer"]["weight_decay"]
    )
    criterion = nn.CrossEntropyLoss()

    # 5) 스케줄러 (LambdaLR with warmup)

    total_epochs  = cfg["epochs"]
    warmup_epochs = cfg["scheduler"]["warmup_epochs"]
    def lr_lambda(epoch):
        if epoch < warmup_epochs:
            return (epoch + 1) / warmup_epochs
        return max(0.0, (total_epochs - epoch) / (total_epochs - warmup_epochs))
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

    model.train()
    return model, optimizer, criterion, scheduler


## <Transformer Encoder 블록>

이번 셀에서는 **ViT-Tiny** 모델의 **Transformer Encoder** 블록을 설명합니다.
비록 코드로 구현하지는 않았지만 입력 패치 임베딩이 어떻게 처리되고, 각각의 블록이 어떤 순서로 구성되는지 확인합니다.

---

### 하나의 Encoder 블록 구성

1. **LayerNorm**
   - **역할**: 입력 임베딩에 대해 정규화 수행 → 학습 안정성 및 빠른 수렴

2. **Multi-Head Self-Attention (MHSA)**
   - **역할**: 입력 패치 간 상호작용을 통해 전역적인 특징(feature) 추출
   - **파라미터**:
     - `embed_dim` → 임베딩 차원 (예: 192)
     - `num_heads` → 어텐션 헤드 수 (예: 2)
     - `qkv_bias` → Q/K/V 생성 시 바이어스 사용 여부

3. **DropPath (Stochastic Depth)**
   - **역할**: 블록 단위 드롭아웃 → 과적합 방지, 모델 일반화 향상
   - **확률**: `drop_path_rate`에 따라 선형적으로 증가

4. **Residual 연결 (Add)**
   - **역할**: 블록 입력과 MHSA+DropPath 출력의 합을 계산 → 기울기 흐름 개선

5. **LayerNorm**
   - **역할**: MLP 블록 입력 정규화

6. **MLP (Feed-Forward Network)**
   - **구성**:
     - `Linear(embed_dim, embed_dim * mlp_ratio)` → 차원 확장
     - `GELU` 활성화 → 비선형성 추가
     - `Linear(embed_dim * mlp_ratio, embed_dim)` → 원래 차원 복원

7. **DropPath** 및 **Residual 연결**
   - MLP 출력에도 DropPath 적용 후 입력과 더함

---

### 블록별 입출력 크기 변화

| 블록 번호 | 입력 패치 수 | 임베딩 차원 | 출력 형태        |
|:---------:|:-----------:|:----------:|:----------------:|
| Block 1   | 65 (1 cls + 64) | 192       | (65, 192)        |
| Block 2 ~ 12 | 65         | 192       | (65, 192)        |

- **cls 토큰 위치**: 첫 번째 패치에 해당하는 벡터 (분류용)  
- **패치 토큰 위치**: 나머지 64개 패치 벡터 (특징 표현)

---

### 전체 Transformer Encoder 쌓기

1. **입력**: 33×33 이미지를 16×16 패치로 분할 → 3×(16×16) 색상
2. **패치 임베딩**: `Linear(3*patch_size*patch_size, embed_dim)` → 192차원 벡터
3. **클래스 토큰** 추가 및 **위치 임베딩** 적용
4. **Encoder 블록** 12회 반복 (ViT-Tiny depth = 12)
5. **LayerNorm** → 최종 `cls` 벡터 추출
6. **MLP 헤드**: `Linear(embed_dim, num_classes)` → 클래스 로짓

## 5. 학습 및 검증 루프 정의

### 손실 함수 & 옵티마이저
- **Criterion**: `nn.CrossEntropyLoss()` → raw logits을 softmax + log로 변환해 클래스 간 차이 측정  
- **Optimizer**: `optim.AdamW()` → weight decay가 적용된 Adam 알고리즘

### 함수 구조
- **train_epoch**(학습 모드)
  1. `model.train()` 활성화 → Dropout/BatchNorm 학습 모드
  2. 배치별 Forward → Loss 계산 → Backward → Optimizer Step
  3. 배치 손실·정확도 누적 → 에포크 평균 반환

- **eval_epoch**(검증 모드)
  1. `model.eval()` & `torch.no_grad()` → 기울기 비활성화
  2. 배치별 Forward → Loss·정확도 누적
  3. 에포크 평균 반환

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct = 0.0, 0
    for x, y in tqdm(loader, desc="Training", leave=True):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x.size(0)
        total_correct += (out.argmax(1) == y).sum().item()
    n = len(loader.dataset)
    return total_loss / n, total_correct / n


def eval_epoch(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct = 0.0, 0
    with torch.no_grad():
        for x, y in tqdm(loader, desc="Validating", leave=True):
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            total_loss += loss.item() * x.size(0)
            total_correct += (out.argmax(1) == y).sum().item()
    n = len(loader.dataset)
    return total_loss / n, total_correct / n

## 6. Main 함수: 전체 파이프라인 실행

다음 코드는 한 번에 전체 학습 과정을 실행하기 위한 주요 단계들을 설명합니다.

1. **설정 로드 & 디바이스 선택**  
   - `vit-tiny-custom.yaml` 파일에서 하이퍼파라미터(`epochs`, `batch_size`, `optimizer` 등)를 불러옵니다.  
   - `torch.device("cuda" if available else "cpu")`를 통해 GPU 혹은 CPU를 선택합니다.  
   - 학습된 체크포인트를 저장·불러올 경로(`ckpt_path`)를 지정합니다.

2. **모델·옵티마이저·스케줄러·손실함수 초기화**  
   - `build_model_components(cfg, device, ckpt_path)` 함수를 호출하여  
     - ViT-Tiny 모델을 생성하고 (`img_size`, `patch_size`, `embed_dim`, `depth` 등 YAML 설정 반영)  
     - 학습된 가중치를 로드하며  
     - Full/Partial fine-tuning 모드를 적용하고  
     - `AdamW` 옵티마이저, `CrossEntropyLoss`, LambdaLR 스케줄러를 반환받습니다.

3. **데이터 분할 및 DataLoader 생성**  
   - `labels.csv`를 읽어와 80:20 비율로 stratified split(`train_labels.csv`, `val_labels.csv`)합니다.  
   - `JetHeatmapDataset` 클래스와 `build_transforms(cfg, train=True/False)`를 이용해 학습 및 검증용 데이터셋을 준비합니다.  
   - `DataLoader`에 `batch_size`, `shuffle`, `num_workers` 설정을 반영해 `train_loader`, `val_loader`를 생성합니다.

4. **학습 루프 실행**  
   - 지정된 `epochs`만큼 반복하며:  
     1. `train_epoch(...)` 호출로 학습 손실과 정확도를 계산  
     2. `eval_epoch(...)` 호출로 검증 손실과 정확도를 계산  
     3. 터미널에  
        ```
        [Epoch/Total] Train Loss: xx.xx, Acc: xx.xx | Val Loss: xx.xx, Acc: xx.xx
        ```  
        형태로 출력  
     4. `scheduler.step()`로 학습률을 업데이트  
     5. 매 에포크마다 `torch.save(...)`로 `model_state_dict`와 `optimizer_state_dict`를 `ckpt_path`에 덮어쓰며 저장

5. **학습 완료**  
   - 모든 에포크가 종료된 후 `"✅ Training complete."` 메시지를 출력하여 학습 과정을 종료합니다.


In [ ]:
# ─────────────────────────────────────
# 설정 & 디바이스
# ─────────────────────────────────────
cfg_path  = "/content/drive/MyDrive/예제/vit-tiny.yaml"
ckpt_path = "/content/drive/MyDrive/예제/checkpoint/timm_vit_finetuned.pth"
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cfg       = yaml.safe_load(open(cfg_path, "r"))

# ─────────────────────────────────────
# 모델·옵티·스케줄러·손실 설정
# ─────────────────────────────────────
model, optimizer, criterion, scheduler = build_model_components(cfg, device, ckpt_path)

# ─────────────────────────────────────
# DataLoader 준비
# ─────────────────────────────────────
base_data = "/content/drive/MyDrive/예제/dataset/MC"
labels_csv = os.path.join(base_data, "labels.csv")
df = pd.read_csv(labels_csv)

# train/val split
tr, va = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
train_csv = os.path.join(base_data, "train_labels.csv")
val_csv   = os.path.join(base_data, "val_labels.csv")
tr.to_csv(train_csv, index=False)
va.to_csv(val_csv, index=False)

# Dataset & DataLoader
train_ds = JetHeatmapDataset(train_csv, base_data, transform=build_transforms(cfg, train=True))
val_ds   = JetHeatmapDataset(val_csv,   base_data, transform=build_transforms(cfg, train=False))

train_loader = DataLoader(
    train_ds,
    batch_size=cfg["batch_size"],
    shuffle=True,
    num_workers=cfg["num_workers"]
)
val_loader = DataLoader(
    val_ds,
    batch_size=cfg["batch_size"],
    shuffle=False,
    num_workers=cfg["num_workers"]
)

# ─────────────────────────────────────
# 학습 루프
# ─────────────────────────────────────
for epoch in range(1, cfg["epochs"] + 1):
    tloss, tacc = train_epoch(model, train_loader, criterion, optimizer, device)
    vloss, vacc = eval_epoch(model, val_loader,   criterion, device)

    print(f"[{epoch}/{cfg['epochs']}] Train {tloss:.4f}/{tacc:.4f} | Val {vloss:.4f}/{vacc:.4f}")

    scheduler.step()
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict()
    }, ckpt_path)

print("✅ Training complete.")



[1/10] Train 0.6957/0.4763 | Val 0.6928/0.5100


[2/10] Train 0.6943/0.4975 | Val 0.6954/0.5100


[3/10] Train 0.6980/0.5112 | Val 0.6943/0.4900


[4/10] Train 0.6983/0.4913 | Val 0.6926/0.5250


[5/10] Train 0.6903/0.5262 | Val 0.6736/0.6550


[6/10] Train 0.6724/0.6375 | Val 0.6305/0.6850


[7/10] Train 0.6683/0.6038 | Val 0.6319/0.6850


[8/10] Train 0.6488/0.6325 | Val 0.5955/0.7050


[9/10] Train 0.6301/0.6587 | Val 0.5691/0.6900


[10/10] Train 0.6220/0.6525 | Val 0.5921/0.7050
✅ Training complete.
